<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект модели классификации позитивных и негативных комментариев

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. Клиенты предлагают свои правки и комментируют изменения других. 

**Задача:** найти инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

**Цель:** найти модель классификации комментариев на позитивные и негативные со значением метрики качества F1 >= 0.75.

**План:**

- Обзор данных.
- Подготовка данных.
- Обучение моделей.

**Описание данных**

Файл - `toxic_comments.csv`. 
- Столбец *text* - текст комментария.
- Столбец *toxic* — целевой признак.

## Подготовка

In [39]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from tqdm.notebook import tqdm
from sklearn.pipeline import Pipeline
from lightgbm import LGBMClassifier

In [2]:
try:
    data = pd.read_csv('toxic_comments.csv', index_col=0)
except:
    data = pd.read_csv('/datasets/toxic_comments.csv', index_col=0)

<div class="alert alert-warning">
<font size="5"><b>Комментарий ревьюера</b></font>

Совет: 


Если не знаешь - чтобы не было столбца  `Unnamed: 0` при чтении файла можно так:


    pd.read_csv(..., index_col=0)

    
(`Unnamed: 0` появляется при не совсем корректном сохранении файла)    

<div>

In [3]:
data.info()
data.sample(15)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


,text,toxic
966,"""Just to add Kansas Bear.When I said:\n\n""""You...",0
84755,edit again so I will have to,0
49509,Re:Move\nI read the arguments and judged that ...,0
144456,"Apology \n\nOk, After a chance meeting with ag...",0
88140,"cited material censorship \nwhy, duncharris an...",0
135735,"June 28, 2005 23:23 (UTC)",0
113678,""" (UTC)\n\n If one takes that attitude, it nev...",0
53146,please reffer tkz.jpg,0
58752,That probably is because you haven't specified...,0
103639,"Fuck's sake, I didn't even edit the article! ...",1


Поверим на пропуски и дубликаты

In [4]:
print('Дубликатов:', data.duplicated().sum())
print('Пропусков:', data.isna().sum())

Дубликатов: 0
Пропусков: text     0
toxic    0
dtype: int64


В данных 159 292 объекта. Пропусков нет, дубликатов нет.

Посмотрим сколько токсичных и не токсичных комментариев

In [5]:
display(data['toxic'].value_counts())

ratio = data['toxic'].value_counts()[0] / data['toxic'].value_counts()[1]
ratio

0    143106
1     16186
Name: toxic, dtype: int64

8.841344371679229

В целевом признаке 90% объектов отрицательного класса.

<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюера</b></font>

Успех:

Данные изучены. Небольшой EDA не помешает, так как это аналитический проект. 


Плюс за

    

-  проверку на сбалансированность 



- Промежуточный вывод в конце раздела


<div class="alert alert-warning">

Совет: 




    
- .sample() вместо .head(), ведь если данные каким то образом упорядоченны, то шансы увидеть что то разнообразное через .sample чуть выше чем через .head (или .tail)     
   







<div class="alert alert-info">
<font size="2"><b>Комментарий студента</b></font>

Готово, спасибо!

<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюераV2</b></font>



Успех 👍:



👍




</div>


Очистим тексты комментариев и леммализируем

<div class="alert alert-danger">
<font size="5"><b>Комментарий ревьюера</b></font>

Ошибка:



- WordNetLemmatizer  рабочий вариант, но у него есть особенности, для корректной работы ему нужно передавать не просто слово, но и POS-тег (Part of Speech, части речи). Набираемся ума-разума [тут](https://webdevblog.ru/podhody-lemmatizacii-s-primerami-v-python/) )  Обрати внимание на функцию `get_wordnet_pos`


- Что насчет токенизации? Лемматизацию мы применяем к словам, а не предложениям, WordNetLemmatizer с предложениями не работает.



И сразу предупрежу что процесс лематизации затянется на полчаса - час. Ниже помог кодом

<div class="alert alert-warning">


Совет: 




- все это можно было сделать с помощью SpaCy лемматизатором и прямо скажем как инструмент он более удобен и универсален, можно поподробней тут [почитать](https://habr.com/ru/post/531940/)  Хотя и более медленный  

<div class="alert alert-info">
<font size="2"><b>Комментарий студента</b></font>

Спасибо!! И за ссылки спасибо! Применим функцию и используем WordNetLemmatizer используем с POS#

Создадим функцию для очистки данных, слов и лемматизируем каждое слово

In [6]:
corpus = list(data['text'])

In [7]:
import nltk
from nltk.stem import WordNetLemmatizer 
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet
import pandas as pd
import re

lemmatizer = WordNetLemmatizer()

def clear_text(text):
    y=re.sub(r"[^'a-zA-Z ]", ' ', text) 
    k=" ".join(y.split())
    return k

from nltk.corpus import wordnet
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Проверим работу

In [9]:
text = "The striped bats are hanging on their feet for best"
print([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)])

['The', 'strip', 'bat', 'be', 'hang', 'on', 'their', 'foot', 'for', 'best']


In [10]:
print(' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)]))

The strip bat be hang on their foot for best


In [13]:
def lemm_(text):

    lemmatized_output = ' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)])     
    return lemmatized_output 

In [14]:
print('было')
sentence1 = "The striped bats are hanging on their feet for best"
sentence2 = "you should be ashamed of yourself went worked"
df_my = pd.DataFrame([sentence1, sentence2])

было


In [15]:
print('стало после лемматизации')
print(df_my[0].apply(lemm_))

стало после лемматизации
0    The strip bat be hang on their foot for best
1       you should be ashamed of yourself go work
Name: 0, dtype: object


Применим к нашим данным

In [18]:
%%time
def lemmafunction(text):
    k=[]
    for i in nltk.word_tokenize(text):
        y=lemmatizer.lemmatize(i, get_wordnet_pos(i))
        k.append(y)
    return ' '.join(k) 

lemy=[]
for i in tqdm(range(len(corpus))):
    
    lemy.append(lemmafunction(clear_text(corpus[i])))
data['lemm_text']=pd.Series(lemy, index=data.index)

  0%|          | 0/159292 [00:00<?, ?it/s]

CPU times: user 19min 27s, sys: 1min 53s, total: 21min 20s
Wall time: 21min 52s


<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюераV2</b></font>



Успех 👍:



Принято.  Но хочу заметить что циклы и Тоне это не самый эффективный вариант.  Использование apply было бы лучше 




</div>


In [22]:
data['text'][2]

"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info."

Преобразуем текст

In [23]:
data['lemm_text'][2]

"Hey man I 'm really not try to edit war It 's just that this guy be constantly remove relevant information and talk to me through edits instead of my talk page He seem to care more about the format than the actual info"

In [24]:
data.head()

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits make under my userna...
1,D'aww! He matches this background colour I'm s...,0,D'aww He match this background colour I 'm see...
2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I 'm really not try to edit war It 's ...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I ca n't make any real suggestion on impr...
4,"You, sir, are my hero. Any chance you remember...",0,You sir be my hero Any chance you remember wha...


<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюера</b></font>

Успех:


- Плюс за использование apply, неэффективные циклы нам ни к чему.


- Да, всегда лучше проверить что получилось  в итоге, так всегда будет возможность поправить ошибку

<div class="alert alert-warning">


Совет: 


    
- попробуй .progress_apply, делает что .apply, но еще и показывает на какой итерации находится процесс.  

Для некоторых версий, чтобы заработал.progress_apply предварительно нужно сделать:
    
    
    from tqdm.notebook import tqdm
    tqdm.pandas()
    

А если делаешь Colab и процесс лемматизации и очистки затягивается, попробуй  .parallel_apply,  кому-то это помогает уменьшить время прогона кода раз в 5-7. Предварительно: 


    
    from pandarallel import pandarallel   
    tqdm.pandas(desc="progress")
    pandarallel.initialize(progress_bar = True)






- после очистки и лемматизации (и убрав стопслова) можно провести частотный анализ текста/[облако слов](https://habr.com/ru/post/517410/) - чтобы получить общее представление о тематике и о наиболее часто встречаемых словах в токсичных и нетоксичных твитах Кроме того графики, рисунки делают проект визуально интересней
    
    
    

<div class="alert alert-info">
<font size="2"><b>Комментарий студента</b></font>

Готово, С tqdm

Разделим выборки. По пропорциям в соответствии распределению по классам в target.

In [26]:
features=data.drop('toxic',axis=1)
target=data['toxic']

In [27]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.25, random_state=42, shuffle=True,stratify=target)

<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюера</b></font>

Успех:


- random_state на месте



- здорово что используешь stratify    



<div class="alert alert-warning">



Совет: 


- если мы используем вручную прописаный цикл, то нам ещё нужен валидационный датасет. 


In [28]:
print(features_train.shape)
print(features_test.shape)
print(target_train.shape)
print(target_test.shape)

(119469, 2)
(39823, 2)
(119469,)
(39823,)


In [29]:
train_sample=features_train.shape[0]/features.shape[0]
test_sample=features_test.shape[0]/features.shape[0]

print('Размер тренировочной выборки- {:.0%}'.format(train_sample))
print('Размер тестовой выборки - {:.0%}'.format(test_sample))

Размер тренировочной выборки- 75%
Размер тестовой выборки - 25%


Распределение класов в тренировочной выборке

In [30]:
ax = target_train.value_counts(normalize=True)
print(ax)

0    0.898384
1    0.101616
Name: toxic, dtype: float64


In [32]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)


[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<div class="alert alert-danger">
<font size="5"><b>Комментарий ревьюера</b></font>

Ошибка:



Да, в тренажере был текст на кирилице, там перевод в unicode оправдан. В нашем случае (латиница) это лишь  увеличит количество потребляемой памяти и это в лучшем случаи, в худшем он обрушает ядро.





</div>

<div class="alert alert-info">
<font size="2"><b>Комментарий студента</b></font>

Да, спасибо, поняла, перевод не оправдан совершенно в данном случае.

<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюераV2</b></font>



Успех 👍:



экономим ресурсы



</div>


<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюера</b></font>

Успех:


Не забыли о стопсловах, они ни к чему и код побежит быстрей

    
<div class="alert alert-warning">


Совет:     

Вопросик:

А стопслова важней убирать  когда мы используем TF-IDF, или когда используе обычный CountVectorizer? 



Итак, подготовили данные для обуения, в том числе:
- Поверили на пропуски и дубликаты.
- Очистили тексты отзывов и леммализировали.
- Получили признаки для обучения, разделили данные на обучающую, тестовую выборки.

<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюера</b></font>



Успех 👍:



Здорово что в каждом разделе оставляешь промежуточные вывод




</div>


<div class="alert alert-info">
<font size="2"><b>Комментарий студента</b></font>

Печально конечно

<div class="alert alert-warning">
<font size="5"><b>Комментарий ревьюераV2</b></font>



Совет 🤔:



Боже мой. Я  использую диктовку с голоса, и он иногда такое пишет...Не заметил, тысяча извинений ) 


## Обучение

<div class="alert alert-danger">
<font size="5"><b>Комментарий ревьюера</b></font>

Ошибка:
 
Во-первых. Работа НЕ выполнена в соответствии с критериями: 



 - модель обучена на обучающем наборе
 - получена оценка качества на валидационном наборе
 - перебор гиперпараметров осуществляется в цикле


Что именно Догадайся сама )    
    
    
Во-вторых
    
    
Пока не выберем лучшую модель на валидации, тестовой выборки для нас как будто бы не существует    

<div class="alert alert-warning">

Совет:
    
    
Советую забыть  вручную прописанные цикл и все модели прогшнать используя  GridSearchCV   


<div class="alert alert-info">
<font size="2"><b>Комментарий студента</b></font>

Обучим модели LogisticRegression и LightGBM.

**LogisticRegression**

In [40]:
lr_pipe = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,3), min_df=3, max_df=0.9, use_idf=1,
               smooth_idf=1, sublinear_tf=1, stop_words=stopwords)),
    ('clf', LogisticRegression(random_state=12345))])

params = {'clf__C': [0.1, 1, 10, 100],
          'clf__class_weight': ['balanced', None]}

lr_grid = GridSearchCV(estimator=lr_pipe, param_grid=params, cv=3, scoring='f1', n_jobs=-1, refit=False)
lr_grid.fit(features_train['text'], target_train)
lr_best_paramms = lr_grid.best_params_

print(lr_best_paramms)
print(lr_grid.best_score_)

/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

{'clf__C': 10, 'clf__class_weight': 'balanced'}
0.7785597134181806


Итак, лучший best_score метрики 0,77

<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюераV2</b></font>



Успех 👍:


    
    
То что ты с ручного цикла сразу перескочила на GS+pipeline, это большой прогресс. Теперь всё по фэншую
    
    

    
<div class="alert alert-warning">


Совет:  



На будущее    
    
    
- Совет 1
    
    
Как создавать собственные функции в pipeline (мы пользовались стандартными из sklearn - Scaler, MinMax, или как в этом проекте TFIDF итп)    


Можешь взять за основу [Ссылка 1](https://dzen.ru/media/id/5ee6f73b7cadb75a66e4c7e3/sozdanie-polzovatelskih-preobrazovatelei-dannyh-62b2a9a80e49941961ffc7a2),
[Ссылка 2](https://towardsdatascience.com/pipelines-custom-transformers-in-scikit-learn-the-step-by-step-guide-with-python-code-4a7d9b068156)

    
    
    
    
- Совет 2  
    
    

Можешь попробовать  feature_engenering (Это когда мы создаём собственные признаки. Во многих случаях это более эффективный способ повысить нашу метрику,  чем долго обучать разные модели) c pipeline:   


    

    
1. Сгенерировать новые фичи, например например посчитать число слов в тексте, длину слов, число знаков препинания, число слов с заглавной итп итд. 
    
   
    
2. Добавить к фичам от векторайзера
    
    
Это можно было реализовать в последующей схеме через [ColumnTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html):
    
    
    new_features = ['lengh', 'number', ....]
    # имена столбцов которых у нас записаны новые фичи (lengh - допустим длина слов твитах, number - количество слов в твите...)
    
    # lemmatized_text - столбец с текстом
    
    features = ColumnTransformer(
                        [("text_preprocess", TfidfVectorizer(stop_words=stopwords), "lemmatized_text"),
                         ("new_features_preprocess", StandardScaler(), new_features)
                        ])

    
    pipe = Pipeline([('features_all_prepross', features),
                     ('model', LogisticRegression(random_state = 42))
                    ])

Какие именно признаки сгенерировать, это целое искусство. Студент который использовал библиотеку
    
    
    from nltk.sentiment.vader import SentimentIntensityAnalyzer    

смог здорово поднять метрику

<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюера</b></font>

Успех:

Корректно использован GridSearch 



- не забыт random_state


- class_weight = 'balanced'


- scoring = 'f1'


 


    

<div class="alert alert-warning">

Совет: 




Молодец что используешь GridSearch, но еще лучше использовать связку GridSearchCV + pipeline. 


О pipeline:

[Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html), это тема которая сразу затрагивает кроссвалидацию, тюнинг "векторайз", подбор гиперпараметров модели и о том что код стоит делать компактным.
    
    
- в TfidfVectorizer(stop_words=stopwords) у тебя по умолчанию ngram_range=(1, 1), тут можно подбирать разное число n- грамм (и другие параметры), максимизируя метрику, но как объединить перебор по ngram_range с обучением моделей, чтобы не делать это по отдельности или с использованием цикла?! pipeline! Готовый [пример для работы с текстами](https://medium.com/@yoni.levine/how-to-grid-search-with-a-pipeline-93147835d916). Всё что нужно там есть, хотя очень лаконично. Можешь погуглить по:


    
    pipeline nlp gridsearchcv



- как избежать ошибки подглядывания в будущее, когда мы предварительно работаем с данными (шкалирование, нормализация, TfidfVectorizer итп итд)? pipeline! особенно это важно, когда мы используем кроссвалидацию. Для TfidfVectorizer делаем .fit (обучаемся) на train, а transform на test, но точно также нужно сделать для валидационной выборки. Но GS делает валидационные внутри себя, спрашивается как добраться до нее и избежать подглядывания в будущее? Казалось бы никак, но нет! Pipeline! ) 
    
    
- pipeline позволяет делать наш код компактней и читабельней, это большой плюс, когда код будет раздуваться     
    
    

         
Если раньше не использовала pipeline то могу посоветовать видео в котором [индус](https://www.youtube.com/watch?v=mOYJCR0IDk8&ab_channel=HimanshuChandra) на английском с сильным акцентом, но на пальцах обьясняет  самое непонятное (по моему опыту): сопряженность методов fit и transform. Там же есть и код и ссылка на текст. Мне помогло )



В общем если сделать GS+pipeline будет вообще хорошо )  
    
<div>   

<div class="alert alert-danger">
<font size="5"><b>Комментарий ревьюера</b></font>



Ошибка ❌:





- Переносим тестирование в самый конец
    
    
- Надо вывести .best_score_ -  Это есть Метрика на валидации когда мы используем GS  
    
    
    

<div class="alert alert-warning">
<font size="5"><b>Комментарий ревьюера</b></font>



Совет 🤔:

Советы по выбору модели для экономии времени   


- Деревянные модели (RF, DT) медленные, и на данном датасете не показывает хорошие результаты. Лучше используй Логистическую регрессию и LightGBM




- Советую сконцентрироваться у Логистической регрессии на переборе "C", а max_iter и solver которые частенько используют, лучше не трогать. 


- Не забываем n_jobs = -1 (Иногда помогает). У LightGBM Можно попробуй явно указать num_threads

**LGBMClassifier**

In [41]:
lgb_pipe = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,3), min_df=3, max_df=0.9, use_idf=1,
               smooth_idf=1, sublinear_tf=1, stop_words=stopwords)),
    ('clf', LGBMClassifier(random_state=12345))])

params = {
  'clf__n_estimators': [50],
  'clf__learning_rate': [0.15, 0.25],
  'clf__max_depth': [6, 8, -1]}

lgb_grid = GridSearchCV(estimator=lgb_pipe, param_grid=params, cv=3, scoring='f1', n_jobs=-1, refit=False)
lgb_grid.fit(features_train['text'], target_train)
lgb_best_params = lgb_grid.best_params_

print(lgb_best_params)
print(lgb_grid.best_score_)

{'clf__learning_rate': 0.25, 'clf__max_depth': -1, 'clf__n_estimators': 50}
0.7553530236307816


Лучший best_score_ метрики у модели LogisticRegression

TF-IDF и финальное тестирование лучшей модели

In [42]:
vectorize = TfidfVectorizer(ngram_range=(1,3),
               min_df=3, max_df=0.9, use_idf=1,
               smooth_idf=1, sublinear_tf=1, stop_words=stopwords)

In [43]:
features_train = vectorize.fit_transform(features_train['text'])
features_test = vectorize.transform(features_test['text'])

In [44]:
lr_t = LogisticRegression(C=10, class_weight='balanced', random_state=12345)
lr_t.fit(features_train, target_train)

/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=10, class_weight='balanced', random_state=12345)




<div class="alert alert-warning">
<font size="5"><b>Комментарий ревьюера</b></font>
    
    
Совет:



    


Не надо воспринимать  GS как способ получить .best_params_, чтобы подставить их в модель и обучить на них. GS это сделал уже и модельку положил тут: .best_estimator_
    
  
То есть вот это не нужно    
    
    lr_t = LogisticRegression(C=10, class_weight='balanced', random_state=12345)
    lr_t.fit(features_train, target_train)

In [46]:
test_pred = lr_t.predict(features_test)
test_f1 = f1_score(target_test, test_pred)
print('F1 на тесте: {:.2f}'.format(test_f1))

F1 на тесте: 0.79


<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюераv2</b></font>

Успех: 



- Если студент получил на тесте f1 выше 0,75, это считается приемлемым результатом.


<div class="alert alert-warning">



Совет: 


Что может помочь добиться лучшего результата (от простого)? 




- можно поиграться [порогом](https://machinelearningmastery.com/threshold-moving-for-imbalanced-classification/)
   



- сгенерировать новые фичи, например  например посчитать число слов в тексте, длину слов итп итд. Или с помощью [тематического моделирования](https://pythobyte.com/python-for-nlp-topic-modeling-8fb3d689/) 
    
    
    



- использование предбученной модели Берта, выбрав соответствующую модель и используя полученные эмбединги, даже на небольшом тренировочном датасете можно обучить модель, которая на test покажет хорошую метрику. В этом случаи можно сразу получить метрику > 0.95 (при правильно выбранной модели)

</div>


<div class="alert alert-danger">
<font size="5"><b>Комментарий ревьюера</b></font>

Ошибка:

А вот тут, в самом конце,  выбрав лучшую модель на валидации, проверяем ее на тестовом датасете - делаем финальное тестирование. И если лучшая модель выбранная на валидационной покажет на test результат хуже требуемого, мы начнем процесс моделирования сначала (а не будем такие - "а давай попробуем на тесте модель которая на валидации не была лучшей, может она нам на test даст нужное качество").         
    
Почему только лучшая?! Это делается для того, чтобы мы даже незначительным образом не "подгонялись" под тестовую выборку. Ведь на train модели обучаются, по валидиации подгоняются гиперпараметры. Эти данные модели "знают". А test (out-of-sample) это уже моделирование прогноза на реальных данных и ситуации когда у нас есть уже лучшая модель (в рельности у нас же не может быть несоклько прогнозов, что то в любом случаи надо выбирать). Вот поэтому такая двухуровневая проверка на подгонку. Кроме того использование мноих моделей с разными гиперпараметрами это тоже подгонка, поэтому выбирая одну и тестируя только ее, мы тем самым боремся с подгонкой через использование многих-многих моделей, когда результат хорош не потому что мы данные почистили хорошо, моделировали правильно итд итп, а потому что из многих моделей хоть какая то случайно "сыграет". 


## Выводы

Итак, в ходе работы, с целью найти инструмент, который будет искать токсичные комментарии и отправлять их на модерацию, была найден модель классификации комментариев на позитивные и негативные со значением метрики качества F1 >= 0.75.

В том числе:

- Подготовлены и проверены данные.
- Очищены от лишних символов, проверены разметка.
- Данные векторизованы методом TF-IDF.
- Разделены на обучающую и тестовую выборки (Размер тренировочной выборки- 75%. Размер тестовой выборки - 25%).
- С гиперпараметрами обучены две модели: LogisticRegression, LGBMClassifier. 

Наилучшей моделью стала LogisticRegression (при: C = 10) со значением F1 = 0.79. Ее можно рекомендовать к использованию. 


<div class="alert alert-info">
<font size="5"><b>Комментарий ревьюера</b></font>



Аня, у тебя старательно выполненная работа, все четко, осмысленно. 


Замечания на будущее:
    


- Где то ты используешь GS, где то вручную прописаный цикл. Зачем? Используй везде GS







Я оставил небольшие советы и вопросики (если есть время и желание можешь воспользоваться/ответить).
    



Обязательное к исправлению:





- WordNetLemmatizer используем с POS - тег  и применяем к словам а не предложениям


    
    


- .astype('U') лишнее, стоит экономить ресурсы, иначе может даже ядро обрушиться




- если захочешь продолжить подбор гиперпараметров в вручную прописанном цикле, тогда тебе потребуется ещё валидационная выборка    


- на test датасете тестируем только лучшую модель (нарушена логика использования датасетов при моделировании)




    

    
    
    
Жду исправлений, для принятия проекта. Если какие то вопросы, то сразу спрашивай ) 


</div>



<div class="alert alert-info">
<font size="5"><b>Комментарий ревьюераV2</b></font>


Ещё раз извиняюсь за "дура" ) Исправил
    
    
Спасибо за работу! Довел до ума pipeline, молодец. 
Красное исправлено. Надеюсь мои советы и вопросики были полезны и в копилочку знаний упало что то новое, а проект стал лучше, и симпатичней.


  
Отличная работа  Аня. Желаю успехов в дальнейшей учебе!


<div class="alert alert-warning">

Совет: 
    
    
Если думаешь и дальше заниматься NLP, то впереди очень современный и модный сейчас подход с использованием эмбедингов от Берт. Вообще именно в НЛП сейчас самые большие прорывы в машинном обучении,  может в курсе про [chatGPT](https://www.youtube.com/watch?v=IMP1zZ9K4Wc&t=3038s), GPT - это братик Берта )

   

<font color='green'><b>Полезные (и просто интересные) материалы:</b> \
Для работы с текстами используют и другие подходы. Например, сейчас активно используются RNN (LSTM) и трансформеры (BERT и другие с улицы Сезам, например, ELMO). НО! Они не являются панацеей, не всегда они нужны, так как и TF-IDF или Word2Vec + модели из классического ML тоже могут справляться. \
BERT тяжелый, существует много его вариаций для разных задач, есть готовые модели, есть надстройки над библиотекой transformers. Если, обучать BERT на GPU (можно в Google Colab или Kaggle), то должно быть побыстрее.\
https://huggingface.co/transformers/model_doc/bert.html \
https://t.me/renat_alimbekov \
https://colah.github.io/posts/2015-08-Understanding-LSTMs/ - Про LSTM \
https://web.stanford.edu/~jurafsky/slp3/10.pdf - про энкодер-декодер модели, этеншены\
https://pytorch.org/tutorials/beginner/transformer_tutorial.html - официальный гайд
по трансформеру от создателей pytorch\
https://transformer.huggingface.co/ - поболтать с трансформером \
Библиотеки: allennlp, fairseq, transformers, tensorflow-text — множествореализованных
методов для трансформеров методов NLP \
Word2Vec https://radimrehurek.com/gensim/models/word2vec.html 


Если понравилась работа с текстами, то можешь посмотреть очень интересный (но очень-очень сложный) курс лекций: https://github.com/yandexdataschool/nlp_course .

Если нравится смотреть и слушать то есть целый курс на Ютубе https://www.youtube.com/watch?v=qDMwIQRQt-M&list=PLEwK9wdS5g0qksxWxtE5c2KuFkIfUXe3i&index=1

</font>


## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны